In [90]:
!touch icf/__init__.py 

In [89]:
%%writefile icf/icf_single.py

from selenium import webdriver
import re
from chromedriver_py import binary_path

def searchInstaUser(user):
    driver = webdriver.Chrome(executable_path=binary_path)
    driver.get('https://www.instagram.com/instagram')
    delay = 3
    driver.implicitly_wait(delay)

    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(user)
    driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[2]/div[2]/div/a[1]/div').click()
    
    r = requests.get(driver.current_url).text
    followers = re.search('"edge_followed_by":{"count":([0-9]+)}',r).group(1)
    
    driver.quit()
    print("{} : {}".format(user, followers))

Writing icf/icf_single.py


In [83]:
searchInstaUser('chris eriksen')

chris eriksen : 2046453


# TODO : error/exception 잡기

In [93]:
%%writefile icf/icf_multi.py

from chromedriver_py import binary_path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from tqdm import tqdm


def searchMultipleInstaUser(userList):

    listUser = []
    listFollower = []
    a=0
    
    while a in tqdm(range(0, int(np.ceil((len(userList))/2)))):
#        print("a = {}".format(a))
        driver = webdriver.Chrome(executable_path=binary_path)
        driver.get('https://www.instagram.com/instagram')
        delay = 3
        driver.implicitly_wait(delay)

        n = 0
        while n < 2:
            
            try:
                userNum = 2*a + n
#                print("userNum: {}+{}={}".format(a,n,userNum))
                user = userList[userNum]
                
                driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input').send_keys(user)
                toNext = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[2]/div[2]/div/a[1]/div')))
                toNext.click()
#                driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[2]/div[2]/div/a[1]/div').click()

                r = requests.get(driver.current_url).text
                followers = re.search('"edge_followed_by":{"count":([0-9]+)}',r).group(1)    
            
                if (r.find('"is_verified":true')!=-1):
#                    print('{} : {}'.format(user, followers))
                    listUser.append(user)
                    listFollower.append(followers)
                else:
#                    print('{} : user not verified'.format(user))
                    listUser.append(user)
                    listFollower.append('user not verified')
            
                n += 1
                continue
                
            except Exception:
                driver.refresh()
                listUser.append(user)
                listFollower.append('Error')
                break
                
            except IndexError:
                driver.quit()
                print("End of List: Terminating Instagram Crawler")
                return
                    
        driver.quit()
        a += 1
    return pd.DataFrame(list(zip(listUser, listFollower)), columns=["Users", "Followers"])

Writing icf/icf_multi.py


In [87]:
searchMultipleInstaUser(ul)

100%|██████████| 3/3 [00:00<00:00, 3008.83it/s]


,Users,Followers
0,chris eriksen,2046455
1,leo messi,156831415
2,cristiano ronaldo,226594585
3,dreandes,user not verified
4,dohyunsra.composer,user not verified
5,dohyunsra.composer,Error


In [94]:
%%writefile icf/__init__.py

__all__ = ["icf_single", "icf_multi"]

Overwriting icf/__init__.py


In [95]:
%%writefile ./setup.py

from setuptools import setup, find_packages

setup(
    name="ICF (Instagram Crawler for Followers)",
    packages=find_packages(),
    include_package_data=True,
    version="0.0.1",
    author="GunhoKo",
    author_email="dreandes@gmail.com",
    zip_safe=False,
)

Writing ./setup.py


In [97]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [98]:
%whos

Interactive namespace is empty.


In [99]:
from icf import *

In [5]:
%whos

Variable     Type      Data/Info
--------------------------------
icf_multi    module    <module 'icf.icf_multi' f<...>wler\\icf\\icf_multi.py'>
icf_single   module    <module 'icf.icf_single' <...>ler\\icf\\icf_single.py'>


In [4]:
from icf import *